In [1]:
import numpy as np
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import math

In [56]:


tickers = ['VTI', 'AGG', 'DBC', '^VIX']
data = yf.download(tickers, start="2006-01-01", end="2020-12-31", interval="1d")['Adj Close']
data_na = data.dropna(axis=0).copy() 

for column in data_na.columns:
    data_na.loc[:, f'{column}_r(t)'] = data_na[column].pct_change()
    data_na.loc[:, f'{column}_r(t+1)'] = data_na[f'{column}_r(t)'].shift(-1)

data_na.dropna(axis=0, inplace=True)
data_na.reset_index(inplace=True)
data_na['Date'] = data_na['Date'].dt.date

data_na


[*********************100%***********************]  4 of 4 completed


Ticker,Date,AGG,DBC,VTI,^VIX,AGG_r(t),AGG_r(t+1),DBC_r(t),DBC_r(t+1),VTI_r(t),VTI_r(t+1),^VIX_r(t),^VIX_r(t+1)
0,2006-02-07,56.438549,20.285254,44.219505,13.590000,-0.000699,-0.000499,-0.028926,-0.004255,-0.009736,0.007194,0.042178,-0.055923
1,2006-02-08,56.410362,20.198935,44.537621,12.830000,-0.000499,0.000600,-0.004255,0.009402,0.007194,-0.001905,-0.055923,0.022603
2,2006-02-09,56.444229,20.388840,44.452785,13.120000,0.000600,-0.002098,0.009402,-0.018205,-0.001905,0.002068,0.022603,-0.019055
3,2006-02-10,56.325829,20.017664,44.544712,12.870000,-0.002098,0.000701,-0.018205,-0.015524,0.002068,-0.004524,-0.019055,0.037296
4,2006-02-13,56.365322,19.706909,44.343201,13.350000,0.000701,-0.001600,-0.015524,-0.008323,-0.004524,0.009486,0.037296,-0.082397
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,2020-12-22,106.759193,13.624199,181.857010,24.230000,0.001443,-0.000678,-0.011004,0.012517,0.000207,0.001710,-0.036963,-0.037969
3746,2020-12-23,106.686798,13.794738,182.168076,23.309999,-0.000678,0.000933,0.012517,0.002747,0.001710,0.001673,-0.037969,-0.076362
3747,2020-12-24,106.786331,13.832635,182.472794,21.530001,0.000933,0.000170,0.002747,-0.006164,0.001673,0.006327,-0.076362,0.007896
3748,2020-12-28,106.804466,13.747366,183.627304,21.700001,0.000170,0.000169,-0.006164,0.002757,0.006327,-0.004174,0.007896,0.063594


In [28]:
def rebalance_portfolio(row, prev_alloc, prices, initial_allocation):
    total_value = (prev_alloc * prices).sum()  # Calcul de la valeur totale du portefeuille
    new_alloc = [(total_value * alloc) / price for alloc, price in zip(initial_allocation, prices)]
    return pd.Series(new_alloc)


In [54]:
def compute_weights(model, alloc_step, period, dataframe, initial_allocation=None):

    if model == "Reallocation":
        # Conversion du tuple en objets datetime
        start_date, end_date = pd.to_datetime(period[0]), pd.to_datetime(period[1])

        # Filtrage du DataFrame selon les dates
        data = dataframe[(dataframe['Date'] >= start_date) & (df['Date'] <= end_date)]

        df_actifs = data[["Date", "AGG", "DBC", "VTI", "^VIX"]]
        portfolio_value = 1

        # Créer un DataFrame pour les allocations initiales
        allocation_df = pd.DataFrame(index=data_na.index, columns=[f'{col}_alloc' for col in test.columns[1:]])

        # Initialiser les allocations du portefeuille pour chaque actif
        initial_prices = data_na.iloc[0, 1:]  # Prix initiaux des actifs
        allocation_df.iloc[0] = [portfolio_value * alloc / price for alloc, price in zip(initial_allocation, initial_prices)]


        # Parcourir chaque date et réallouer le portefeuille selon la période définie
        for i in range(1, len(test)):
            current_date = pd.to_datetime(test.loc[i, 'Date'])
            previous_date = pd.to_datetime(test.loc[i - 1, 'Date'])
            
            # Vérifier le changement de période pour la réallocation
            if (period == 'year' and current_date.year != previous_date.year) or \
            (period == 'month' and (current_date.year != previous_date.year or current_date.month != previous_date.month)) or \
            (period == 'day' and current_date != previous_date):
                
                previous_alloc = allocation_df.iloc[i - 1].values.astype(float)
                current_prices = test.iloc[i, 1:].values.astype(float)
                
                # Réallocation basée sur les prix actuels et les allocations initiales
                allocation_df.iloc[i] = rebalance_portfolio(test.iloc[i], previous_alloc, current_prices, initial_allocation)
            
            else:
                # Conserver la même allocation
                allocation_df.iloc[i] = allocation_df.iloc[i - 1]
            
            # Mise à jour de la valeur du portefeuille
            portfolio_df.loc[i, 'portfolio_value'] = (allocation_df.iloc[i] * test.iloc[i, 1:].values).sum()


        # Déterminer la première année de la période
        first_date = pd.to_datetime(test['Date'].iloc[0])
        first_year = first_date.year

        # Initialisation du DataFrame des poids
        weights_df = pd.DataFrame(index=test.index, columns=test.columns[1:])

        # Poids fixes pour la première année
        weights_first_year = initial_allocation  

        # Assignation des poids pour la première année et les années suivantes
        for i in range(len(test)):
            current_year = pd.to_datetime(test.loc[i, 'Date']).year
            
            if current_year == first_year:
                weights_df.loc[i] = weights_first_year  
            elif current_year > first_year:
                # Si c'est la première occurrence de l'année
                if i == 0 or pd.to_datetime(test.loc[i - 1, 'Date']).year != current_year:
                    # Récupérer les prix de la première ligne de l'année
                    first_row_prices = test.iloc[i, 1:].values.astype(float)
                    # Calculer les poids basés sur la première ligne de l'année
                    weights_df.loc[i] = fixed_weights = first_row_prices / first_row_prices.sum()
                else:
                    # Pour les autres jours de la même année, conserver les mêmes poids
                    weights_df.loc[i] = weights_df.loc[i - 1]

    return weights_df

In [55]:
period = ("2011-01-01", "2020-04-30")

model="Reallocation"
alloc_step="year"
initial_allocation=[0.25, 0.25, 0.25, 0.25]

weights_realloc = compute_weights(model, alloc_step, period, data_na, initial_allocation)


TypeError: Cannot compare Timestamp with datetime.date. Use ts == pd.Timestamp(date) or ts.date() == date instead.

In [ ]:
weights_realloc

In [17]:
# Allocation initiale
initial_allocation = [0.25, 0.25, 0.25, 0.25]  # Allocations initiales pour VTI, AGG, DBC, VIX
portfolio_value = 1
test = data_na[["Date", "AGG", "DBC", "VTI", "^VIX"]]

In [18]:
# Créer un DataFrame pour les allocations initiales
allocation_df = pd.DataFrame(index=data_na.index, columns=[f'{col}_alloc' for col in test.columns[1:]])

# Initialiser les allocations du portefeuille pour chaque actif
initial_prices = data_na.iloc[0, 1:]  # Prix initiaux des actifs
allocation_df.iloc[0] = [portfolio_value * alloc / price for alloc, price in zip(initial_allocation, initial_prices)]


In [21]:
# DataFrame pour stocker les valeurs du portefeuille
portfolio_df = pd.DataFrame(index=data_na.index, columns=['Date', 'portfolio_value'])
portfolio_df['Date'] = data_na['Date']

In [34]:
# Paramètre de période de réallocation : 'day', 'month', ou 'year'
rebalancing_period = 'day'  # Change à 'day', 'month', ou 'year' selon la fréquence souhaitée


In [35]:

# Parcourir chaque date et réallouer le portefeuille selon la période définie
for i in range(1, len(test)):
    current_date = pd.to_datetime(test.loc[i, 'Date'])
    previous_date = pd.to_datetime(test.loc[i - 1, 'Date'])
    
    # Vérifier le changement de période pour la réallocation
    if (rebalancing_period == 'year' and current_date.year != previous_date.year) or \
       (rebalancing_period == 'month' and (current_date.year != previous_date.year or current_date.month != previous_date.month)) or \
       (rebalancing_period == 'day' and current_date != previous_date):
        
        previous_alloc = allocation_df.iloc[i - 1].values.astype(float)
        current_prices = test.iloc[i, 1:].values.astype(float)
        
        # Réallocation basée sur les prix actuels et les allocations initiales
        allocation_df.iloc[i] = rebalance_portfolio(test.iloc[i], previous_alloc, current_prices, initial_allocation)
    
    else:
        # Conserver la même allocation
        allocation_df.iloc[i] = allocation_df.iloc[i - 1]
    
    # Mise à jour de la valeur du portefeuille
    portfolio_df.loc[i, 'portfolio_value'] = (allocation_df.iloc[i] * test.iloc[i, 1:].values).sum()


In [29]:
# Parcourir chaque année et réallouer le portefeuille au 1er janvier de chaque année
for i in range(1, len(test)):
    current_year = pd.to_datetime(test.loc[i, 'Date']).year
    previous_year = pd.to_datetime(test.loc[i-1, 'Date']).year
    
    if current_year != previous_year:  # Réallocation si on change d'année
        previous_alloc = allocation_df.iloc[i-1].values.astype(float)
        current_prices = test.iloc[i, 1:].values.astype(float)
        
        # Réallocation basée sur les prix actuels et les allocations initiales
        allocation_df.iloc[i] = rebalance_portfolio(test.iloc[i],previous_alloc, current_prices, initial_allocation)
    else:
        allocation_df.iloc[i] = allocation_df.iloc[i-1]  # Conserver la même allocation
    
    # Utilisation de loc pour éviter les avertissements
    portfolio_df.loc[i, 'portfolio_value'] = (allocation_df.iloc[i] * test.iloc[i, 1:].values).sum()


In [38]:
# Déterminer la première année de la période
first_date = pd.to_datetime(test['Date'].iloc[0])
first_year = first_date.year

# Initialisation du DataFrame des poids
weights_df = pd.DataFrame(index=test.index, columns=test.columns[1:])

# Poids fixes pour la première année
weights_first_year = initial_allocation  

# Assignation des poids pour la première année et les années suivantes
for i in range(len(test)):
    current_year = pd.to_datetime(test.loc[i, 'Date']).year
    
    if current_year == first_year:
        weights_df.loc[i] = weights_first_year  
    elif current_year > first_year:
        # Si c'est la première occurrence de l'année
        if i == 0 or pd.to_datetime(test.loc[i - 1, 'Date']).year != current_year:
            # Récupérer les prix de la première ligne de l'année
            first_row_prices = test.iloc[i, 1:].values.astype(float)
            # Calculer les poids basés sur la première ligne de l'année
            weights_df.loc[i] = fixed_weights = first_row_prices / first_row_prices.sum()
        else:
            # Pour les autres jours de la même année, conserver les mêmes poids
            weights_df.loc[i] = weights_df.loc[i - 1]


In [11]:
# Supprimer la dernière ligne du DataFrame des poids
weights_df = weights_df.drop(weights_df.index[-1]).reset_index(drop=True)


In [12]:
# Calculer les rendements quotidiens
returns_df = data_na.iloc[:, 1:].pct_change().fillna(0)
returns_df['Date'] = data_na['Date']
returns_df = returns_df[['Date'] + list(returns_df.columns[:-1])]



In [13]:
returns_df = returns_df.drop(index=returns_df.index[0]).reset_index(drop=True)

In [14]:
returns_df = returns_df.drop(columns=['Date'])

In [15]:
# Calculer le produit terme à terme
weighted_returns = returns_df.values * weights_df.values

# Somme sur les lignes pour obtenir le rendement du portefeuille
portfolio_returns = weighted_returns.sum(axis=1)

In [16]:
portfolio_returns

array([-0.004203606759099488, -0.00439414757280957, 0.002763499438648076,
       ..., 0.003776011157360968, 0.00011878580811951986,
       0.01359452227762845], dtype=object)

In [17]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_returns.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_returns.std()
std_dev = std_dev * math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1334
Standard déviation : 0.1133
Sharpe ratio : 1.1769


In [18]:
# Initialisation des listes pour stocker les premières données de chaque année
weights_first_list = []
prices_first_list = []
returns_first_list = []
allocation_first_list = []

# Boucle sur les années de 2011 à 2020
for year in range(2011, 2021):
    # Filtrer les données pour l'année en question
    mask = pd.to_datetime(data_na['Date']).dt.year == year
    
    # Stocker la première ligne de chaque année dans les listes
    weights_first_list.append(weights_df.loc[mask].iloc[0])
    prices_first_list.append(data_na.loc[mask].iloc[0])
    returns_first_list.append(returns_df.loc[mask].iloc[0])
    allocation_first_list.append(allocation_df.loc[mask].iloc[0])

# Convertir les listes en DataFrames
weights_first_df = pd.DataFrame(weights_first_list).reset_index(drop=True)
prices_first_df = pd.DataFrame(prices_first_list).reset_index(drop=True)
returns_first_df = pd.DataFrame(returns_first_list).reset_index(drop=True)
allocation_first_df = pd.DataFrame(allocation_first_list).reset_index(drop=True)


## Prise en compte du cout de transaction

In [19]:
C = 0.01  

# Initialisation du vecteur des coûts de transaction
transaction_costs = np.zeros(len(weights_df))

# Calcul des coûts de transaction
for t in range(2, len(weights_df)):
    # Calcul des différences de poids
    diff_weights = weights_df.iloc[t - 1].values - weights_df.iloc[t - 2].values
    
    # Somme des valeurs absolues des différences
    sum_abs_diff = np.sum(np.abs(diff_weights))
    
    # Coût de transaction pour le temps t
    transaction_costs[t] = C * sum_abs_diff



In [20]:
portfolio_with_transac_cost = portfolio_returns +transaction_costs

In [21]:
# Calcul de l'espérance des rendements (moyenne)
expected_return = portfolio_with_transac_cost.mean()
expected_return = (expected_return+1)**252-1

# Calcul de la standard déviation
std_dev = portfolio_with_transac_cost.std()
std_dev = std_dev * math.sqrt(252)
risk_free_rate = 0.0

# Calcul du Sharpe ratio
sharpe_ratio = (expected_return - risk_free_rate) / std_dev

# Affichage des résultats
print(f"Espérance des rendements : {expected_return:.4f}")
print(f"Standard déviation : {std_dev:.4f}")
print(f"Sharpe ratio : {sharpe_ratio:.4f}")


Espérance des rendements : 0.1347
Standard déviation : 0.1133
Sharpe ratio : 1.1883


## Prise en compte de la volatilité scaling

In [22]:
# Initialisation du DataFrame pour stocker les volatilités ex-ante
volatility_df = pd.DataFrame(index=returns_df.index, columns=returns_df.columns)

# Boucle sur chaque colonne (actif) pour calculer la volatilité
for col in returns_df.columns:
    # Calcul de la variance exponentiellement pondérée sur 50 jours
    ew_var = returns_df[col].ewm(span=50, adjust=False).var()
    
    # Calcul de la volatilité ex-ante en prenant la racine carrée de la variance
    volatility_df[col] = ew_var.shift(1).apply(lambda x: x**0.5)



In [23]:
volatility_df.iloc[48:53]

Ticker,AGG,DBC,VTI,^VIX
48,0.002047,0.010439,0.008735,0.071994
49,0.002005,0.012417,0.008787,0.076133
50,0.002024,0.012272,0.009203,0.084170
51,0.001995,0.013688,0.009374,0.085929
52,0.001960,0.013469,0.009250,0.085940


In [24]:
# Définition du paramètre alpha
vol_target = 0.1  

vol_scale_df = vol_target / volatility_df


In [25]:
vol_scale_df

Ticker,AGG,DBC,VTI,^VIX
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,27.668754,5.893858,18.905163,18.480298
3,39.136400,8.331284,26.660422,5.377897
4,40.391862,9.540802,32.571576,6.568140
...,...,...,...,...
2340,10.199396,4.962047,2.627129,0.794350
2341,10.400440,5.061499,2.680198,0.809760
2342,10.605831,5.145042,2.726392,0.805257
2343,10.747953,5.196251,2.770256,0.815172
